In [1]:
import pyspark
sc.setLogLevel("ERROR")

In [2]:
df = spark.read.csv('*.csv', header='true')

In [3]:
print(type(df))
df.printSchema()

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- Province_State: string (nullable = true)
 |-- Country_Region: string (nullable = true)
 |-- Last_Update: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long_: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- Incident_Rate: string (nullable = true)
 |-- People_Tested: string (nullable = true)
 |-- People_Hospitalized: string (nullable = true)
 |-- Mortality_Rate: string (nullable = true)
 |-- UID: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Testing_Rate: string (nullable = true)
 |-- Hospitalization_Rate: string (nullable = true)



In [4]:
df.createOrReplaceTempView("covid")

In [5]:
SQL = """
SELECT Province_State
          , CAST(SUM(Recovered) AS INT) Total_Recovered
          , CAST(SUM(People_Hospitalized) AS INT) AS Total_Hospitalized
          , CAST(SUM(Deaths) AS INT) AS Total_Deaths
FROM covid 
WHERE Province_State != "Recovered"
GROUP BY Province_State
ORDER BY Total_Hospitalized DESC
LIMIT 10
"""

df1 = spark.sql(SQL)
df1.show(10, False)

+--------------+---------------+------------------+------------+
|Province_State|Total_Recovered|Total_Hospitalized|Total_Deaths|
+--------------+---------------+------------------+------------+
|New York      |140922         |179990            |63435       |
|New Jersey    |60376          |39694             |30027       |
|Florida       |null           |29530             |6952        |
|Massachusetts |null           |22682             |16147       |
|Georgia       |null           |21875             |5589        |
|Maryland      |9961           |21666             |6365        |
|Connecticut   |16106          |20536             |8642        |
|Virginia      |16103          |17805             |3503        |
|Ohio          |null           |15585             |5681        |
|Indiana       |68994          |14089             |5264        |
+--------------+---------------+------------------+------------+



In [6]:
SQL = """
SELECT     
    CAST(
          SUM(Deaths) * 100 /  
          ( SELECT SUM(Deaths) AS total FROM covid )          
          AS DECIMAL(8,2) 
        )         
    AS New_York_Deaths_Percent
FROM covid 
WHERE Province_State == "New York"
"""
df2 = spark.sql(SQL)
df2.show(10, False)

+-----------------------+
|New_York_Deaths_Percent|
+-----------------------+
|25.02                  |
+-----------------------+

